<a href="https://colab.research.google.com/github/hjfuentes/Laboratorio-Datos-Sociales/blob/main/Talleres/Taller_Portafolio_MonteCarlo_Sharpe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller: Portafolio con Walmart (WMT), Bank of New York Mellon (BK) y Cintas (CTAS)


Conoce nuestra [aplicación web](https://ratiosharpe-7pvzekscctfzmiumsxlfa7.streamlit.app/)

### Datos, retornos, correlación, portafolio y simulación Monte Carlo (Sharpe Ratio)

> **Objetivo del taller:** construir, paso a paso, un mini–pipeline de análisis de portafolio:  
1) descargar datos
2) calcular retornos
3) visualizar
4) correlación
5) portafolio (retorno & riesgo),  
6) Sharpe Ratio
7) simulación Monte Carlo de pesos para encontrar combinaciones atractivas.

**Acciones del caso:** WMT, BK y CTAS  
**Nota:** Este notebook es didáctico: buscamos claridad y trazabilidad (código entendible), no “magia” ni atajos.

## 0) ¿Qué necesitamos instalar e importar?

Usaremos `yfinance` para precios y `plotly` para gráficos interactivos.

- Si ejecutas esto en Colab: probablemente ya tienes Plotly.
- Si te falta `yfinance`, instálalo con `pip` (solo una vez).

In [ ]:
# Si no tienes yfinance instalado, descomenta:
# !pip -q install yfinance

import numpy as np
import pandas as pd
import yfinance as yf

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
pd.set_option("display.float_format", lambda x: f"{x:,.6f}")

## 1) ¿Cómo obtenemos datos financieros?

La idea es simple: para cada ticker descargamos **Close** (precio de cierre).

> En análisis de portafolio, trabajar con **precios de cierre** evita distorsiones cuando comparamos retornos en el tiempo.

In [ ]:
tickers = ["WMT", "BK", "CTAS"]
start = "2022-01-01"   # puedes cambiar el rango
end = "2026-02-12"            # None = hasta hoy

In [ ]:
data = yf.download(tickers, start=start, end=end, progress=False)[["Close"]]
prices = data["Close"].dropna()

/tmp/ipython-input-3241815116.py:1: FutureWarning:

YF.download() has changed argument auto_adjust default to True



In [ ]:
prices.head()

Ticker,BK,CTAS,WMT
Date,,,
2022-01-03,51.788837,102.185745,45.765411
2022-01-04,53.638748,102.431526,44.926979
2022-01-05,53.267006,97.822098,45.534439
2022-01-06,54.258347,97.104050,45.407890
2022-01-07,55.170025,95.759537,45.841347


### ¿Cómo se ven los precios?

Un gráfico rápido para ver tendencia general.

In [ ]:
fig = px.line(prices, title="Precios ajustados (Adj Close)")
fig.update_layout(legend_title_text="", margin=dict(l=20, r=20, t=40, b=20))
fig.show()

## 2) ¿Cómo medimos el retorno? usando `pct_change`

El retorno diario simple se calcula como:

$$
R_t = \frac{P_t - P_{t-1}}{P_{t-1}}
$$

En Pandas, esto es `pct_change()`.

> Trabajaremos con retornos diarios por simplicidad. Luego, anualizaremos cuando tenga sentido (Sharpe, volatilidad, etc.).

In [ ]:
returns = prices.pct_change().dropna()
returns.head()

Ticker,BK,CTAS,WMT
Date,,,
2022-01-04,0.035720,0.002405,-0.018320
2022-01-05,-0.006930,-0.045000,0.013521
2022-01-06,0.018611,-0.007340,-0.002779
2022-01-07,0.016803,-0.013846,0.009546
2022-01-10,0.007541,0.004554,-0.001933


### ¿Cómo se distribuyen los retornos?

Una visualización rápida: series de retornos diarios (pueden verse “ruidosas”, es normal).

In [ ]:
fig = px.line(returns, title="Retornos diarios (pct_change)")
fig.update_layout(legend_title_text="", margin=dict(l=20, r=20, t=40, b=20))
fig.show()

### ¿Podemos ver el retorno acumulado?

El retorno acumulado (crecimiento de 1 dólar) es:

$$
(1+R_1)(1+R_2)\dots(1+R_t)
$$

Esto nos permite comparar desempeño de forma visual.

In [ ]:
growth = (1 + returns).cumprod()

fig = px.line(growth, title="Crecimiento de 1 unidad (retorno acumulado)")
fig.update_layout(legend_title_text="", margin=dict(l=20, r=20, t=40, b=20))
fig.show()

## 3) ¿Cómo estimamos retorno esperado y riesgo esperado?

En un enfoque práctico, una aproximación común es:

- **Retorno esperado** ≈ promedio histórico de retornos.
- **Riesgo** ≈ desviación estándar de retornos (volatilidad).

Para comparaciones anuales usamos anualización con ~252 días de trading.

$$
\mu_{ann} \approx 252\,\bar{R}, \qquad \sigma_{ann} \approx \sqrt{252}\,\sigma
$$

In [ ]:
trading_days = 252

mean_daily = returns.mean()
vol_daily = returns.std()

mean_ann = mean_daily * trading_days
vol_ann = vol_daily * np.sqrt(trading_days)

summary_assets = pd.DataFrame({
    "Mean_Daily": mean_daily,
    "Vol_Daily": vol_daily,
    "Mean_Annual": mean_ann,
    "Vol_Annual": vol_ann
})

summary_assets

,Mean_Daily,Vol_Daily,Mean_Annual,Vol_Annual
Ticker,,,,
BK,0.000952,0.015620,0.239894,0.247954
CTAS,0.000755,0.014207,0.190261,0.225535
WMT,0.001100,0.013804,0.277310,0.219129


### Visualización rápida: retorno vs volatilidad

Esto nos da una intuición de *riesgo–retorno* por activo (no es optimización aún, solo un mapa).

In [ ]:
scatter_df = summary_assets.reset_index().rename(columns={"index": "Ticker"})

fig = px.scatter(
    scatter_df,
    x="Vol_Annual",
    y="Mean_Annual",
    text="Ticker",
    title="Riesgo–Retorno (anualizado)"
)
fig.update_traces(textposition="top center")
fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
fig.show()

## 4) ¿Qué es la correlación y por qué importa?

La correlación mide qué tan “juntos” se mueven dos activos.

- Correlación **alta** → diversifica menos.
- Correlación **baja/negativa** → diversifica más.

En portafolios, no solo importa el riesgo individual, sino cómo se **combinan** (covarianzas).

In [ ]:
corr = returns.corr()
corr

Ticker,BK,CTAS,WMT
Ticker,,,
BK,1.000000,0.432449,0.220924
CTAS,0.432449,1.000000,0.350671
WMT,0.220924,0.350671,1.000000


### Visualización: mapa de correlación (heatmap)

Ligero y directo.

In [ ]:
fig = px.imshow(
    corr,
    text_auto=True,
    title="Correlación de retornos (diarios)"
)
fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
fig.show()

## 5) ¿Qué es un portafolio y cómo calculamos su retorno esperado?

Un portafolio asigna **pesos** a cada activo.

Ejemplo:
- WMT 40%, BK 20%, CTAS 40%

Si $w$ es el vector de pesos y $\mu$ el vector de retornos esperados (anuales), el retorno esperado del portafolio es:

$$
\mu_p = \sum_i w_i\mu_i
$$

En código: `mu_p = weights @ mean_ann`

In [ ]:
weights = np.array([0.4, 0.2, 0.4])  # WMT, BK, CTAS
weights.sum()

np.float64(1.0)

In [ ]:
mu_p = weights @ mean_ann.values
mu_p

np.float64(0.24493409142958533)

## 6) ¿Cómo calculamos el riesgo del portafolio?

La volatilidad (riesgo) del portafolio depende de la **matriz de covarianzas**.

Si $\Sigma$ es la covarianza anualizada:

$$
\sigma_p = \sqrt{w^T \Sigma w}
$$

> Este es el corazón de Markowitz: el riesgo no se suma linealmente, se combina vía covarianzas.

In [ ]:
cov_ann = returns.cov() * trading_days

sigma_p = np.sqrt(weights.T @ cov_ann.values @ weights)
sigma_p

np.float64(0.17331419658847783)

## 7) ¿Qué es el Sharpe Ratio y por qué lo usamos?

El Sharpe Ratio mide retorno **ajustado al riesgo**:

$$
Sharpe = \frac{\mu_p - r_f}{\sigma_p}
$$

- $\mu_p$: retorno esperado anual del portafolio
- $\sigma_p$: volatilidad anual del portafolio
- $r_f$: tasa libre de riesgo (anual)

> **Interpretación rápida:** a mayor Sharpe, “mejor” compensación retorno/riesgo (comparando bajo el mismo $r_f$).

In [ ]:
rf = 0.03  # 3% anual (supuesto simple para el taller)

sharpe_p = (mu_p - rf) / sigma_p
sharpe_p

np.float64(1.2401412905598899)

## 8) Mini–simulación Monte Carlo (pesos aleatorios)

Ahora viene la parte entretenida: generar pesos al azar y calcular, para cada combinación:

- retorno esperado del portafolio
- riesgo (volatilidad)
- Sharpe Ratio

Así exploramos **muchas** combinaciones posibles.

### 8.1) Primero: un ejemplo “uno por uno” (sin bucle)

Creamos pesos aleatorios y los normalizamos para que sumen 1.

In [ ]:
w = np.random.rand(len(tickers))
w = w / w.sum()

w, w.sum()

(array([0.0886406 , 0.48823372, 0.42312568]), np.float64(1.0))

In [ ]:
mu_rand = w @ mean_ann.values
sigma_rand = np.sqrt(w.T @ cov_ann.values @ w)
sharpe_rand = (mu_rand - rf) / sigma_rand

mu_rand, sigma_rand, sharpe_rand

(np.float64(0.23149346376087782),
 np.float64(0.1770848550397764),
 np.float64(1.1378356648038526))

### 8.2) Ahora sí: muchas combinaciones (con bucle)

Generaremos `N` portafolios aleatorios y guardaremos sus métricas en un DataFrame.

> Importante: esto **no** es optimización exacta; es una exploración por muestreo. Para taller es perfecto porque se entiende y es visual.

In [ ]:
N = 50000

results = []
for _ in range(N):
    w = np.random.rand(len(tickers))
    w = w / w.sum()

    mu = w @ mean_ann.values
    sigma = np.sqrt(w.T @ cov_ann.values @ w)
    sharpe = (mu - rf) / sigma

    results.append([mu, sigma, sharpe, *w])

cols = ["Return", "Vol", "Sharpe"] + [f"w_{t}" for t in tickers]
mc = pd.DataFrame(results, columns=cols)

mc.sort_values('Sharpe',ascending=False).head()

,Return,Vol,Sharpe,w_WMT,w_BK,w_CTAS
43535,0.258646,0.177463,1.288416,0.324395,0.074978,0.600626
14147,0.258795,0.177579,1.288414,0.324456,0.073241,0.602303
43777,0.258384,0.177261,1.288404,0.323769,0.078254,0.597977
34363,0.258935,0.177689,1.288398,0.321466,0.072919,0.605615
33488,0.258275,0.177178,1.288395,0.321253,0.080588,0.598159


### ¿Cuál fue el mejor portafolio (por Sharpe) en esta simulación?

Buscamos el máximo Sharpe dentro de nuestras combinaciones aleatorias.

In [ ]:
best = mc.loc[mc["Sharpe"].idxmax()]
best

,43535
Return,0.258646
Vol,0.177463
Sharpe,1.288416
w_WMT,0.324395
w_BK,0.074978
w_CTAS,0.600626


### Visualización: nube de portafolios (Return vs Vol)

- Cada punto es un portafolio (pesos diferentes).
- El color representa Sharpe.
- Marcamos el mejor Sharpe encontrado.

*(Gráfico simple, interactivo, sin sobrecarga.)*

In [ ]:
fig = px.scatter(
    mc,
    x="Vol",
    y="Return",
    color="Sharpe",
    title="Monte Carlo de portafolios (WMT, BK, CTAS)"
)

# marcar el mejor
fig.add_trace(
    go.Scatter(
        x=[best["Vol"]],
        y=[best["Return"]],
        mode="markers",
        marker=dict(size=12, symbol="x"),
        name="Mejor Sharpe"
    )
)

fig.update_layout(legend_title_text="", margin=dict(l=20, r=20, t=40, b=20))
fig.show()

## 9) Cierre: ¿Qué aprendimos?

En este notebook hicimos un flujo completo y entendible:

1) **Datos**: descargamos precios ajustados con `yfinance`.  
2) **Retornos**: calculamos retornos diarios con `pct_change()`.  
3) **Riesgo–retorno**: anualizamos medias y volatilidades.  
4) **Correlación**: entendimos diversificación (no todo es “riesgo individual”).  
5) **Portafolio**: calculamos retorno y volatilidad con pesos.  
6) **Sharpe**: comparamos combinaciones con una métrica riesgo–ajustada.  
7) **Monte Carlo**: exploramos miles de portafolios con pesos aleatorios.